<a href="https://colab.research.google.com/github/DLProjectTextGeneration/TextGeneration/blob/main/Projet_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import PowerTransformer, MinMaxScaler

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# **Importation des données:**

TU peux direct importer le fichier dans l'onglet fichier à droite du code

J'ai modifié le texte original: 


*   j'ai retiré les noms de chapitres
*   J'ai retiré tout ce qui n'était pas l'histoire (crédits, titre, table des matières...)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving britney-spears.txt to britney-spears.txt


In [ ]:
data = open('alice.txt','r')
alice = data.read()
data.close()

In [ ]:
print(alice)

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her own mind (as well as she could, for the
hot day made her feel very sleepy and stupid), whether the pleasure of
making a daisy-chain would be worth the trouble of getting up and
picking the daisies, when suddenly a White Rabbit with pink eyes ran
close by her.

There was nothing so _very_ remarkable in that; nor did Alice think it
so _very_ much out of the way to hear the Rabbit say to itself, “Oh
dear! Oh dear! I shall be late!” (when she thought it over afterwards,
it occurred to her that she ought to have wondered at this, but at the
time it all seemed quite natural); but when the Rabbit actually _took a
watch out of its waistcoat-pocket_, and looked at it

# **Data cleaning :**

J'ai pris les fonctions du cours que j'ai un peu modifié pour nettoyer le texte:

In [ ]:
def convert_text_to_lowercase(df):
    df = df.lower()
    return df
    
def not_regex(pattern):
        return r"((?!{}).)".format(pattern)

def remove_punctuation(df):
    #df = df.replace('\n', ' ')
    df = " ".join(df.split())
    df = df.replace('_', '')
    alphanumeric_characters_extended = '(\\b[-/]\\b|[a-zA-Z0-9])'
    df = df.replace(not_regex(alphanumeric_characters_extended), ' ')
    return df


def text_cleaning(df):
    """
    Takes in a string of text, then performs the following:
    1. convert text to lowercase
    2. remove punctuation and new line characters '\n'
    3. Tokenize sentences
    4. Remove all stopwords
    5. convert tokenized text to text
    """
    df = convert_text_to_lowercase(df)
    df = remove_punctuation(df)
    return df

In [ ]:
alice=str(alice)
alice = text_cleaning(alice)

In [ ]:
alice

'alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought alice “without pictures or conversations?” so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her. there was nothing so very remarkable in that; nor did alice think it so very much out of the way to hear the rabbit say to itself, “oh dear! oh dear! i shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and t

In [ ]:
characters = sorted(list(set(alice)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

vocab_size = len(characters)
print('Number of unique characters: ', vocab_size)
print(characters)

Number of unique characters:  44
[' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ù', '—', '‘', '’', '“', '”']


In [ ]:
X = []   # extracted sequences
Y = []   # the target - the follow up character
length = len(alice)
seq_length = 100   #number of characters to consider before predicting the following character

In [ ]:
for i in range(0, length - seq_length, 1):
    sequence = alice[i:i + seq_length]
    label = alice[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
    
print('Number of extracted sequences:', len(X))

Number of extracted sequences: 141590


In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [ ]:
X_modified.shape, Y_modified.shape

((141590, 100, 1), (141590, 44))

In [ ]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

Pour reprendre le dernier modèle qu'on a train. Ici on reprend epoch = 06 et loss = 1.1442

In [ ]:
filename = "/content/baseline-improvement-06-0.9963.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

OSError: ignored

Créer un checkpoint à chaque epoch.

In [ ]:
filepath="/content/training_checkpoints/baseline-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X_modified, Y_modified, epochs=30, batch_size=128, callbacks = callbacks_list)

Epoch 1/30
1107/1107 [==============================] - 83s 67ms/step - loss: 2.9813
Epoch 2/30
1107/1107 [==============================] - 79s 71ms/step - loss: 2.4737
Epoch 3/30
1107/1107 [==============================] - 79s 71ms/step - loss: 2.1809
Epoch 4/30
1107/1107 [==============================] - 79s 71ms/step - loss: 1.9863
Epoch 5/30
1107/1107 [==============================] - 78s 71ms/step - loss: 1.8453
Epoch 6/30
1107/1107 [==============================] - 79s 71ms/step - loss: 1.7374
Epoch 7/30
1107/1107 [==============================] - 79s 71ms/step - loss: 1.6478
Epoch 8/30
1107/1107 [==============================] - 79s 71ms/step - loss: 1.5765
Epoch 9/30
1107/1107 [==============================] - 79s 71ms/step - loss: 1.5158
Epoch 10/30
1107/1107 [==============================] - 79s 71ms/step - loss: 1.4563
Epoch 11/30
1107/1107 [==============================] - 79s 71ms/step - loss: 1.4056
Epoch 12/30
1107/1107 [==============================] - 79s 71

# **Generating text**

In [ ]:
start = np.random.randint(0, len(X)-1) # or generate random start

string_mapped = list(X[start])

full_string = [n_to_char[value] for value in string_mapped]

print("Seed:")
print("\"", ''.join(full_string), "\"")

Seed:
" akes rather a handsome pig, i think.” and she began thinking over other children she knew, who might "


In [ ]:
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])
    
    string_mapped.append(pred_index)  # add the predicted character to the end
    string_mapped = string_mapped[1:len(string_mapped)] # shift the string one character forward by removing pos. 0

In [ ]:
# combining text
txt=""
for char in full_string:
    txt = txt+char

In [ ]:
print(start)
print(txt)

66127
akes rather a handsome pig, i think.” and she began thinking over other children she knew, who might do very seadi to say which) and the words dame very soon and a cods of sole withe gead of hearts, and she stoee ruite sure the world gear off simence. and the whole party think it was very uile, and she went on talking as it went, and some of the shale of the court was a warch to tay when she had been anything to she baby oo the stoppess, and the torn of things aack to the thmee of the court, and


# Test avec une autre base

In [ ]:
data = open('britney-spears.txt','r')
britney = data.read()
data.close()

In [ ]:
print(britney)

They say get ready for the revolution
I think it's time we find some sorta solution
Somebody's caught up in the endless pollution
They need to wake up, stop living illusions I know you need to hear this
Why won't somebody feel this
This is my wish that we all feel connected
This is my wish that nobodies neglected Be like a rocket baby
Be like a rocket Take off
Just fly, away (ay, ay)
To find your space Take off
Just fly, away (ay, ay)
To find your place Take off You know what they say about mixing the races
And in the end we got the same faces
My mama told me got love yourself first
And if you disagree, get off this damn earth I want to feel connected
Don't want to be neglected
This is my wish that we all find our places
This is my wish that we all escalate (yeah) Be like a rocket baby
Be like a rocket Take off
Just fly, away (ay, ay)
To find your space Take off
Just fly, away (ay, ay)
To find your space Take off
Just fly, away (ay, ay)
To find your space Take off
Just fly, away (ay, a

In [ ]:
britney=str(britney)
britney = text_cleaning(britney)

In [ ]:
britney

'they say get ready for the revolution i think it\'s time we find some sorta solution somebody\'s caught up in the endless pollution they need to wake up, stop living illusions i know you need to hear this why won\'t somebody feel this this is my wish that we all feel connected this is my wish that nobodies neglected be like a rocket baby be like a rocket take off just fly, away (ay, ay) to find your space take off just fly, away (ay, ay) to find your place take off you know what they say about mixing the races and in the end we got the same faces my mama told me got love yourself first and if you disagree, get off this damn earth i want to feel connected don\'t want to be neglected this is my wish that we all find our places this is my wish that we all escalate (yeah) be like a rocket baby be like a rocket take off just fly, away (ay, ay) to find your space take off just fly, away (ay, ay) to find your space take off just fly, away (ay, ay) to find your space take off just fly, away (

In [ ]:
characters = sorted(list(set(britney)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

vocab_size = len(characters)
print('Number of unique characters: ', vocab_size)
print(characters)

Number of unique characters:  50
[' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '6', '7', '8', ':', ';', '?', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
X = []   # extracted sequences
Y = []   # the target - the follow up character
length = len(britney)
seq_length = 100   #number of characters to consider before predicting the following character

In [ ]:
for i in range(0, length - seq_length, 1):
    sequence = britney[i:i + seq_length]
    label = britney[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])
    
print('Number of extracted sequences:', len(X))

Number of extracted sequences: 135186


In [ ]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [ ]:
X_modified.shape, Y_modified.shape

((135186, 100, 1), (135186, 50))

In [ ]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
filename = "/content/training_checkpoints/baseline-improvement-britney-06-0.8624.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
filepath="/content/training_checkpoints/baseline-improvement-britney-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X_modified, Y_modified, epochs=6, batch_size=128, callbacks = callbacks_list)

Epoch 1/6
1057/1057 [==============================] - 76s 70ms/step - loss: 0.7952

Epoch 00001: loss improved from inf to 0.80679, saving model to /content/training_checkpoints/baseline-improvement-britney-01-0.8068.hdf5
Epoch 2/6
1057/1057 [==============================] - 74s 70ms/step - loss: 0.7506

Epoch 00002: loss improved from 0.80679 to 0.76305, saving model to /content/training_checkpoints/baseline-improvement-britney-02-0.7630.hdf5
Epoch 3/6
1057/1057 [==============================] - 74s 70ms/step - loss: 0.7042

Epoch 00003: loss improved from 0.76305 to 0.71494, saving model to /content/training_checkpoints/baseline-improvement-britney-03-0.7149.hdf5
Epoch 4/6
1057/1057 [==============================] - 74s 70ms/step - loss: 0.6701

Epoch 00004: loss improved from 0.71494 to 0.68573, saving model to /content/training_checkpoints/baseline-improvement-britney-04-0.6857.hdf5
Epoch 5/6
1057/1057 [==============================] - 74s 70ms/step - loss: 0.6363

Epoch 00005

In [ ]:
start = np.random.randint(0, len(X)-1) # or generate random start

string_mapped = list(X[start])

full_string = [n_to_char[value] for value in string_mapped]

print("Seed:")
print("\"", ''.join(full_string), "\"")

Seed:
" someone else boxer in the ring trying to defend myself in the private eye to see what's going on (th "


In [ ]:
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])
    
    string_mapped.append(pred_index)  # add the predicted character to the end
    string_mapped = string_mapped[1:len(string_mapped)] # shift the string one character forward by removing pos. 0

In [ ]:
# combining text
txt=""
for char in full_string:
    txt = txt+char

In [ ]:
print(start)
print(txt)

87449
someone else boxer in the ring trying to defend myself in the private eye to see what's going on (the gooos of your lept sore tomethmes i run so fonna be alright but me i don't keed you to make me move like i'm gosta ge saadn cnr you to fane the solene i was born to make you happy i dan tell her now wou fou a boyt to sooi a was beliers fant i got the sters phat i cream that whip, everybody goin' trip just like a circus don't stand there wat something i wish there was something i wish there was s
